In [ ]:
import netCDF4
import numpy as np
import pandas as pd
from scipy.interpolate import interp2d

# Pfad zur deiner .nc-Datei
nc_file_path = '../../Data/JPL_Mascons_Canada_Interpolated_linear_modes2-5.nc'

# Name der Variablen, die die Maskendaten enthält (musst du eventuell anpassen)
mask_var_name = "lwe_thickness"

# Ziel-Grid-Dimensionen (5x5 Grad)
target_lon_start = -141.0
target_lon_end = -52.0
target_lat_start = 41.0
target_lat_end = 83.0
n_lon_target = int((target_lon_end - target_lon_start) / 5) + 1
n_lat_target = int((target_lat_end - target_lat_start) / 5) + 1

# Dateiname für die Ausgabe-CSV-Datei
output_csv_file = 'Mascons_Canada_grid_5x5.csv'

try:
    # .nc-Datei öffnen
    with netCDF4.Dataset(nc_file_path, 'r') as nc_file:


        # Lese die Maskendaten und die Koordinaten (Namen können variieren)
        mask_data = nc_file.variables[mask_var_name][:]

        # Annahme: Die Koordinaten sind 'lon' und 'lat'. Passe ggf. an.
        original_lon = nc_file.variables['lon'][:]
        original_lat = nc_file.variables['lat'][:]

    # Erstelle das Ziel-Grid
    target_lon = np.linspace(target_lon_start, target_lon_end, n_lon_target)
    target_lat = np.linspace(target_lat_start, target_lat_end, n_lat_target)

    # Erstelle eine 2D-Interpolationsfunktion (bilinear)
    interpolate_function = interp2d(original_lon, original_lat, mask_data, kind='linear')

    # Interpoliere die Daten auf das neue Grid
    interpolated_data = interpolate_function(target_lon, target_lat)

    # Erstelle einen Pandas DataFrame
    df = pd.DataFrame(interpolated_data, index=target_lat, columns=target_lon)
    df.index.name = 'latitude'
    df.columns.name = 'longitude'

    # Speichere den DataFrame als CSV-Datei
    df.to_csv(output_csv_file)

    print(f"Die interpolierten Daten wurden erfolgreich in '{output_csv_file}' gespeichert.")

except FileNotFoundError:
    print(f"Fehler: Die Datei '{nc_file_path}' wurde nicht gefunden.")
except KeyError as e:
    print(f"Fehler: Die Variable '{e}' wurde in der .nc-Datei nicht gefunden. Bitte überprüfe die Variablennamen.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

Variablen in der Datei:
Fehler: Die Variable ''lwe_thickness'' wurde in der .nc-Datei nicht gefunden. Bitte überprüfe die Variablennamen.
